# Aula 1 

In [2]:
# Pacotes
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

In [3]:
# Inicia sessao spark
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_01')
    .getOrCreate()
)

In [4]:
df = spark.read.csv('wc2018-players.csv', header = True, inferSchema = True)

In [5]:
df.show(3)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 3 rows



In [6]:
# Verifica tipo das colunas
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



In [7]:
# Renomeia coluna
df = df.withColumnRenamed('Pos.', 'posicao')\
.withColumnRenamed('Team', 'selecao')\
.withColumnRenamed('#', 'numero')\
.withColumnRenamed('FIFA Popular Name', 'nm_fifa')\
.withColumnRenamed('Birth Date', 'dt_nasc')\
.withColumnRenamed('Shirt Name', 'nm_camisa')\
.withColumnRenamed('Club', 'time')\
.withColumnRenamed('Height', 'altura')\
.withColumnRenamed('Weight', 'peso')

In [8]:
df.show(3)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+
|  selecao|numero|posicao|           nm_fifa|   dt_nasc| nm_camisa|                time|altura|peso|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|  66|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+
only showing top 3 rows



In [9]:
# Nomes das colunas
df.columns

['selecao',
 'numero',
 'posicao',
 'nm_fifa',
 'dt_nasc',
 'nm_camisa',
 'time',
 'altura',
 'peso']

In [10]:
# Conta valores nulos em cada coluna
for i in df.columns:
    print(i, df.filter(df[i].isNull()).count())


selecao 0
numero 0
posicao 0
nm_fifa 0
dt_nasc 0
nm_camisa 0
time 0
altura 0
peso 0


In [11]:
# Selecionar colunas
df.select('selecao', 'nm_camisa').show(3)

+---------+----------+
|  selecao| nm_camisa|
+---------+----------+
|Argentina|TAGLIAFICO|
|Argentina|     PAVÓN|
|Argentina|   LANZINI|
+---------+----------+
only showing top 3 rows



In [12]:
# Selecionar colunas (2)
df.select(col('selecao'), col('nm_camisa')).show(3)


+---------+----------+
|  selecao| nm_camisa|
+---------+----------+
|Argentina|TAGLIAFICO|
|Argentina|     PAVÓN|
|Argentina|   LANZINI|
+---------+----------+
only showing top 3 rows



In [13]:
# Filtro
df.filter(col('selecao') == 'Brazil').show(10)

+-------+------+-------+-----------------+----------+-----------+--------------------+------+----+
|selecao|numero|posicao|          nm_fifa|   dt_nasc|  nm_camisa|                time|altura|peso|
+-------+------+-------+-----------------+----------+-----------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|13.01.1988|     TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|04.05.1985|FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|11.06.1989|     FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|05.02.1992|  NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|12.06.1992|P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|14.09.1990|   D. COSTA|   Juventus FC (ITA)|   182|  70|
| Brazil| 

In [14]:
# Filtro com AND (&)
df.filter((col('selecao') == 'Argentina') & (col('altura') > 180) & (col('peso') >= 85)).show()

+---------+------+-------+--------------+----------+---------+--------------------+------+----+
|  selecao|numero|posicao|       nm_fifa|   dt_nasc|nm_camisa|                time|altura|peso|
+---------+------+-------+--------------+----------+---------+--------------------+------+----+
|Argentina|     6|     DF|FAZIO Federico|17.03.1987|    FAZIO|       AS Roma (ITA)|   199|  85|
|Argentina|    12|     GK| ARMANI Franco|16.10.1986|   ARMANI|CA River Plate (ARG)|   189|  85|
|Argentina|     1|     GK| GUZMAN Nahuel|10.02.1986|   GUZMÁN|   Tigres UANL (MEX)|   192|  90|
+---------+------+-------+--------------+----------+---------+--------------------+------+----+



In [15]:
# Filtro equivalente ao AND (&) 
df.filter(col('selecao') == 'Brazil').filter(col('altura') < 170).show()

+-------+------+-------+-------+----------+---------+--------------------+------+----+
|selecao|numero|posicao|nm_fifa|   dt_nasc|nm_camisa|                time|altura|peso|
+-------+------+-------+-------+----------+---------+--------------------+------+----+
| Brazil|    18|     MF|   FRED|05.03.1993|     FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    22|     DF| FAGNER|11.06.1989|   FAGNER|SC Corinthians (BRA)|   168|  67|
+-------+------+-------+-------+----------+---------+--------------------+------+----+



In [16]:
# Filtro OR ( | )
df.filter((col('nm_camisa') == 'FRED') | (col('nm_camisa') == 'MESSI')).show()

+---------+------+-------+------------+----------+---------+--------------------+------+----+
|  selecao|numero|posicao|     nm_fifa|   dt_nasc|nm_camisa|                time|altura|peso|
+---------+------+-------+------------+----------+---------+--------------------+------+----+
|Argentina|    10|     FW|MESSI Lionel|24.06.1987|    MESSI|  FC Barcelona (ESP)|   170|  72|
|   Brazil|    18|     MF|        FRED|05.03.1993|     FRED|FC Shakhtar Donet...|   169|  64|
+---------+------+-------+------------+----------+---------+--------------------+------+----+



In [17]:
# Criar novas colunas
df = df.withColumn('data_ano', lit(2018))

In [18]:
df.show(3)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+
|  selecao|numero|posicao|           nm_fifa|   dt_nasc| nm_camisa|                time|altura|peso|data_ano|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|  65|    2018|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|  66|    2018|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+
only showing top 3 rows



In [19]:
# Cria coluna com a funcao substring
df.withColumn('selecao_abrev', substring('selecao', 1, 3)).show(3) # pega do primeiro ate o terceiro elemento da string

+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+-------------+
|  selecao|numero|posicao|           nm_fifa|   dt_nasc| nm_camisa|                time|altura|peso|data_ano|selecao_abrev|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+-------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|          Arg|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|  65|    2018|          Arg|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|  66|    2018|          Arg|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+-------------+
only showing top 3 rows



In [20]:
df.printSchema()

root
 |-- selecao: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- posicao: string (nullable = true)
 |-- nm_fifa: string (nullable = true)
 |-- dt_nasc: string (nullable = true)
 |-- nm_camisa: string (nullable = true)
 |-- time: string (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: integer (nullable = true)
 |-- data_ano: integer (nullable = false)



In [21]:
# coverte tipo da coluna
df = df.withColumn('data_ano', col('data_ano').cast(IntegerType()))

In [22]:
df.printSchema()

root
 |-- selecao: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- posicao: string (nullable = true)
 |-- nm_fifa: string (nullable = true)
 |-- dt_nasc: string (nullable = true)
 |-- nm_camisa: string (nullable = true)
 |-- time: string (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: integer (nullable = true)
 |-- data_ano: integer (nullable = false)



# Aula 2

In [24]:
df = df.withColumn('dt_nasc', to_date('dt_nasc', 'dd.mm.yyyy')) # formata coluna para o padrao yyyy-mm-dd
df = df.withColumn('dt_nasc', col('dt_nasc').cast(DateType())) # converte para o tipo data

In [25]:
df.show(3)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+
|  selecao|numero|posicao|           nm_fifa|   dt_nasc| nm_camisa|                time|altura|peso|data_ano|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|1992-01-31|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|
|Argentina|    22|     MF|    PAVON Cristian|1996-01-21|     PAVÓN|CA Boca Juniors (...|   169|  65|    2018|
|Argentina|    15|     MF|    LANZINI Manuel|1993-01-15|   LANZINI|West Ham United F...|   167|  66|    2018|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+
only showing top 3 rows



In [26]:
df.printSchema()

root
 |-- selecao: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- posicao: string (nullable = true)
 |-- nm_fifa: string (nullable = true)
 |-- dt_nasc: date (nullable = true)
 |-- nm_camisa: string (nullable = true)
 |-- time: string (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: integer (nullable = true)
 |-- data_ano: integer (nullable = false)



In [27]:
# ROW_NUMBER()
w_func = Window.partitionBy('selecao').orderBy(desc('peso'))
df.withColumn('rn', row_number().over(w_func)).show(30)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+---+
|  selecao|numero|posicao|           nm_fifa|   dt_nasc| nm_camisa|                time|altura|peso|data_ano| rn|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+---+
|Argentina|     1|     GK|     GUZMAN Nahuel|1986-01-10|    GUZMÁN|   Tigres UANL (MEX)|   192|  90|    2018|  1|
|Argentina|     6|     DF|    FAZIO Federico|1987-01-17|     FAZIO|       AS Roma (ITA)|   199|  85|    2018|  2|
|Argentina|    12|     GK|     ARMANI Franco|1986-01-16|    ARMANI|CA River Plate (ARG)|   189|  85|    2018|  3|
|Argentina|    16|     DF|       ROJO Marcos|1990-01-20|      ROJO|Manchester United...|   189|  82|    2018|  4|
|Argentina|     2|     DF|   MERCADO Gabriel|1987-01-18|   MERCADO|    Sevilla FC (ESP)|   181|  81|    2018|  5|
|Argentina|    17|     DF|  OTAMENDI Nicolas|1988-01-12|  OTAMENDI|Manchester City F...|

In [28]:
# RANK()
w_func = Window.partitionBy('selecao').orderBy(desc('peso'))
df.withColumn('rn', rank().over(w_func)).show()

+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+---+
|  selecao|numero|posicao|           nm_fifa|   dt_nasc| nm_camisa|                time|altura|peso|data_ano| rn|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+---+
|Argentina|     1|     GK|     GUZMAN Nahuel|1986-01-10|    GUZMÁN|   Tigres UANL (MEX)|   192|  90|    2018|  1|
|Argentina|     6|     DF|    FAZIO Federico|1987-01-17|     FAZIO|       AS Roma (ITA)|   199|  85|    2018|  2|
|Argentina|    12|     GK|     ARMANI Franco|1986-01-16|    ARMANI|CA River Plate (ARG)|   189|  85|    2018|  2|
|Argentina|    16|     DF|       ROJO Marcos|1990-01-20|      ROJO|Manchester United...|   189|  82|    2018|  4|
|Argentina|     2|     DF|   MERCADO Gabriel|1987-01-18|   MERCADO|    Sevilla FC (ESP)|   181|  81|    2018|  5|
|Argentina|    17|     DF|  OTAMENDI Nicolas|1988-01-12|  OTAMENDI|Manchester City F...|

In [29]:
# DANSE_RANK()
w_func = Window.partitionBy('selecao').orderBy('peso')
df.withColumn('rn', dense_rank().over(w_func)).show()

+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+---+
|  selecao|numero|posicao|           nm_fifa|   dt_nasc| nm_camisa|                time|altura|peso|data_ano| rn|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|1992-01-31|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|  1|
|Argentina|    22|     MF|    PAVON Cristian|1996-01-21|     PAVÓN|CA Boca Juniors (...|   169|  65|    2018|  1|
|Argentina|    15|     MF|    LANZINI Manuel|1993-01-15|   LANZINI|West Ham United F...|   167|  66|    2018|  2|
|Argentina|    18|     DF|    SALVIO Eduardo|1990-01-13|    SALVIO|    SL Benfica (POR)|   167|  69|    2018|  3|
|Argentina|    10|     FW|      MESSI Lionel|1987-01-24|     MESSI|  FC Barcelona (ESP)|   170|  72|    2018|  4|
|Argentina|     4|     DF|  ANSALDI Cristian|1986-01-20|   ANSALDI|     Torino FC (ITA)|

In [30]:
# media de altura por grupo
df.groupBy('selecao').agg(avg('altura')).orderBy(avg('altura'), ascending = False).show(3)

+-------+------------------+
|selecao|       avg(altura)|
+-------+------------------+
| Serbia|186.69565217391303|
|Denmark| 186.6086956521739|
|Germany| 185.7826086956522|
+-------+------------------+
only showing top 3 rows



In [31]:
# maximo de altura por grupo
df.groupBy('selecao').agg(max('altura')).orderBy(max('altura'), ascending = False).show(3) # max por grupo

+-------+-----------+
|selecao|max(altura)|
+-------+-----------+
|Croatia|        201|
|Denmark|        200|
|Belgium|        199|
+-------+-----------+
only showing top 3 rows



In [32]:
# describe
df.describe(['altura']).show() 

+-------+-----------------+
|summary|           altura|
+-------+-----------------+
|  count|              736|
|   mean|182.4076086956522|
| stddev|6.930924233929302|
|    min|              165|
|    max|              201|
+-------+-----------------+



In [33]:
# sumarisa df
w_func = Window.partitionBy('selecao').orderBy(desc('altura'))
w_func_2 = Window.partitionBy('selecao')

df.withColumn('rn', row_number().over(w_func))\
.withColumn('media_altura', avg('altura').over(w_func_2))\
.withColumn('min_altura', min('altura').over(w_func_2))\
.withColumn('max_altura', max('altura').over(w_func_2))\
.select('selecao','media_altura', 'min_altura', 'max_altura')\
.filter(col('rn') == 1)\
.orderBy('altura', ascending = False).show()

+--------------+------------------+----------+----------+
|       selecao|      media_altura|min_altura|max_altura|
+--------------+------------------+----------+----------+
|       Croatia| 185.2608695652174|       172|       201|
|       Denmark| 186.6086956521739|       171|       200|
|     Argentina|178.43478260869566|       167|       199|
|       Belgium|185.34782608695653|       169|       199|
|       Iceland|185.52173913043478|       170|       198|
|        Sweden| 185.7391304347826|       177|       198|
|        France|183.30434782608697|       168|       197|
|Korea Republic| 181.8695652173913|       170|       197|
|       Nigeria|184.52173913043478|       172|       197|
|        Panama|182.17391304347825|       165|       197|
|    Costa Rica|180.69565217391303|       172|       196|
|       England| 182.7391304347826|       170|       196|
|        Russia| 184.3913043478261|       173|       196|
|       Senegal|183.65217391304347|       173|       196|
|       Urugua

# Aula 3

In [35]:
df.show(3)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+
|  selecao|numero|posicao|           nm_fifa|   dt_nasc| nm_camisa|                time|altura|peso|data_ano|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|1992-01-31|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|
|Argentina|    22|     MF|    PAVON Cristian|1996-01-21|     PAVÓN|CA Boca Juniors (...|   169|  65|    2018|
|Argentina|    15|     MF|    LANZINI Manuel|1993-01-15|   LANZINI|West Ham United F...|   167|  66|    2018|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+
only showing top 3 rows



In [36]:
# DISTINCT()
df.select('selecao').distinct().show()

+--------------+
|       selecao|
+--------------+
|        Russia|
|       Senegal|
|        Sweden|
|       IR Iran|
|       Germany|
|        France|
|     Argentina|
|       Belgium|
|          Peru|
|       Croatia|
|       Nigeria|
|Korea Republic|
|         Spain|
|       Denmark|
|       Morocco|
|        Panama|
|       Iceland|
|       Uruguay|
|        Mexico|
|       Tunisia|
+--------------+
only showing top 20 rows



In [37]:
# COLLECT() --> transforma o resultado do comando em uma lista
lista = df.select('selecao').distinct().collect()
print(lista)

[Row(selecao='Russia'), Row(selecao='Senegal'), Row(selecao='Sweden'), Row(selecao='IR Iran'), Row(selecao='Germany'), Row(selecao='France'), Row(selecao='Argentina'), Row(selecao='Belgium'), Row(selecao='Peru'), Row(selecao='Croatia'), Row(selecao='Nigeria'), Row(selecao='Korea Republic'), Row(selecao='Spain'), Row(selecao='Denmark'), Row(selecao='Morocco'), Row(selecao='Panama'), Row(selecao='Iceland'), Row(selecao='Uruguay'), Row(selecao='Mexico'), Row(selecao='Tunisia'), Row(selecao='Saudi Arabia'), Row(selecao='Switzerland'), Row(selecao='Brazil'), Row(selecao='Japan'), Row(selecao='England'), Row(selecao='Poland'), Row(selecao='Portugal'), Row(selecao='Australia'), Row(selecao='Costa Rica'), Row(selecao='Egypt'), Row(selecao='Serbia'), Row(selecao='Colombia')]


In [38]:
# extraindo somente os nomes dos paises da lista

lista_2 = []

for i in lista:
    lista_2.append(i[0])

print(lista_2)

['Russia', 'Senegal', 'Sweden', 'IR Iran', 'Germany', 'France', 'Argentina', 'Belgium', 'Peru', 'Croatia', 'Nigeria', 'Korea Republic', 'Spain', 'Denmark', 'Morocco', 'Panama', 'Iceland', 'Uruguay', 'Mexico', 'Tunisia', 'Saudi Arabia', 'Switzerland', 'Brazil', 'Japan', 'England', 'Poland', 'Portugal', 'Australia', 'Costa Rica', 'Egypt', 'Serbia', 'Colombia']


In [39]:
# WHEN e OTHERWISE
df.withColumn('regiao', when(col('selecao') == 'Argentina', 'Sulamerica')\
.when(col('selecao') == 'Brazil', 'Sulameria').otherwise('Outros')).show(30)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+----------+
|  selecao|numero|posicao|           nm_fifa|   dt_nasc| nm_camisa|                time|altura|peso|data_ano|    regiao|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+--------+----------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|1992-01-31|TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|Sulamerica|
|Argentina|    22|     MF|    PAVON Cristian|1996-01-21|     PAVÓN|CA Boca Juniors (...|   169|  65|    2018|Sulamerica|
|Argentina|    15|     MF|    LANZINI Manuel|1993-01-15|   LANZINI|West Ham United F...|   167|  66|    2018|Sulamerica|
|Argentina|    18|     DF|    SALVIO Eduardo|1990-01-13|    SALVIO|    SL Benfica (POR)|   167|  69|    2018|Sulamerica|
|Argentina|    10|     FW|      MESSI Lionel|1987-01-24|     MESSI|  FC Barcelona (ESP)|   170|  72|    2018|Sulamerica|
|Argentina|     4|     DF|  ANSA

In [40]:
df.select('selecao').distinct().show(32)

+--------------+
|       selecao|
+--------------+
|        Russia|
|       Senegal|
|        Sweden|
|       IR Iran|
|       Germany|
|        France|
|     Argentina|
|       Belgium|
|          Peru|
|       Croatia|
|       Nigeria|
|Korea Republic|
|         Spain|
|       Denmark|
|       Morocco|
|        Panama|
|       Iceland|
|       Uruguay|
|        Mexico|
|       Tunisia|
|  Saudi Arabia|
|   Switzerland|
|        Brazil|
|         Japan|
|       England|
|        Poland|
|      Portugal|
|     Australia|
|    Costa Rica|
|         Egypt|
|        Serbia|
|      Colombia|
+--------------+



In [41]:
# Cria lista continentes para cada pais
europa = ['Russia', 'Sweden', 'Germany', 'France', 'Belgium', 'Croatia', 'Spain', 'Denmark', 'Iceland', 'Switzerland', 'England', 'Poland', 'Portugal', 'Serbia']
africa = ['Senegal', 'Nigeria', 'Morocco', 'Tunisia', 'Egypt']
asia = ['IR Iran', 'Korea Republic', 'Saudi Arabia', 'Japan']
oceania = ['Australia']
america = ['Argentina', 'Peru', 'Panama', 'Uruguay', 'Mexico', 'Brazil', 'Costa Rica', 'Colombia']

In [42]:
# ISIN()
# Cria coluna classificando cada pais por continente
df_2 = df.withColumn('continente', when(col('selecao').isin(europa), 'europa')\
.when(col('selecao').isin(africa), 'africa')\
.when(col('selecao').isin(asia), 'asia')\
.when(col('selecao').isin(oceania), 'oceania')\
.when(col('selecao').isin(america), 'america'))

In [43]:
df_2.select('selecao', 'continente').distinct().show(32)

+--------------+----------+
|       selecao|continente|
+--------------+----------+
|        Brazil|   america|
|        France|    europa|
|       Croatia|    europa|
|       Iceland|    europa|
|         Spain|    europa|
|        Poland|    europa|
|        Serbia|    europa|
|       Tunisia|    africa|
|      Portugal|    europa|
|  Saudi Arabia|      asia|
|       Belgium|    europa|
|       England|    europa|
|       Uruguay|   america|
|      Colombia|   america|
|   Switzerland|    europa|
|         Egypt|    africa|
|       IR Iran|      asia|
|     Argentina|   america|
|Korea Republic|      asia|
|       Nigeria|    africa|
|        Sweden|    europa|
|        Russia|    europa|
|     Australia|   oceania|
|       Morocco|    africa|
|        Mexico|   america|
|    Costa Rica|   america|
|          Peru|   america|
|       Germany|    europa|
|        Panama|   america|
|       Denmark|    europa|
|       Senegal|    africa|
|         Japan|      asia|
+--------------+----

In [44]:
# separa em dois dataframes --> fazendo isso para testar o comando UNION()
df_america =  df_2.filter(col('continente') == 'america')
df_asia = df_2.filter(col('continente') == 'asia')

In [45]:
df_america.select('selecao').distinct().show()

+----------+
|   selecao|
+----------+
| Argentina|
|      Peru|
|    Panama|
|   Uruguay|
|    Mexico|
|    Brazil|
|Costa Rica|
|  Colombia|
+----------+



In [46]:
df_asia.select('selecao').distinct().show()

+--------------+
|       selecao|
+--------------+
|       IR Iran|
|Korea Republic|
|  Saudi Arabia|
|         Japan|
+--------------+



In [47]:
# UNION()
df_union = df_america.union(df_asia)

In [48]:
df_union.select('selecao').distinct().show()

+--------------+
|       selecao|
+--------------+
|     Argentina|
|          Peru|
|        Panama|
|       Uruguay|
|        Mexico|
|        Brazil|
|    Costa Rica|
|      Colombia|
|       IR Iran|
|Korea Republic|
|  Saudi Arabia|
|         Japan|
+--------------+



In [49]:
# Cria dois dataframes para testas o comando JOIN()
df_br = df.select('selecao', 'numero', 'posicao', 'nm_camisa', 'altura')\
.filter(col('selecao') == 'Brazil')

df_arg = df.select('selecao', 'numero', 'posicao', 'nm_camisa', 'altura')\
.filter(col('selecao') == 'Argentina')

In [50]:
print(df_br.count(), df_arg.count())

23 23


In [51]:
df_br.join(df_arg, df_br.numero == df_arg.numero,'left').show()

+-------+------+-------+-----------+------+---------+------+-------+----------+------+
|selecao|numero|posicao|  nm_camisa|altura|  selecao|numero|posicao| nm_camisa|altura|
+-------+------+-------+-----------+------+---------+------+-------+----------+------+
| Brazil|    18|     MF|       FRED|   169|Argentina|    18|     DF|    SALVIO|   167|
| Brazil|    21|     FW|     TAISON|   172|Argentina|    21|     FW|    DYBALA|   177|
| Brazil|    17|     MF|FERNANDINHO|   179|Argentina|    17|     DF|  OTAMENDI|   181|
| Brazil|    22|     DF|     FAGNER|   168|Argentina|    22|     MF|     PAVÓN|   169|
| Brazil|    10|     FW|  NEYMAR JR|   175|Argentina|    10|     FW|     MESSI|   170|
| Brazil|    11|     MF|P. COUTINHO|   172|Argentina|    11|     MF|  DI MARÍA|   178|
| Brazil|     7|     FW|   D. COSTA|   182|Argentina|     7|     MF|    BANEGA|   175|
| Brazil|     6|     DF|FILIPE LUIS|   182|Argentina|     6|     DF|     FAZIO|   199|
| Brazil|     9|     FW|   G. JESUS|   175|

In [52]:
# retona o que nao foi encontrado no df_arg, nesse caso nenhum
df_br.join(df_arg, df_br.numero == df_arg.numero,'anti').show()

+-------+------+-------+---------+------+
|selecao|numero|posicao|nm_camisa|altura|
+-------+------+-------+---------+------+
+-------+------+-------+---------+------+

